In [ ]:
import pandas as pd 
df = pd.read_csv('instruments.csv')

In [ ]:
NIFTY_50 = 25000
level = 20
strikes = []
for i in range(0,level+1):
     strikes.append(NIFTY_50 + (i * 50))
     strikes.append(NIFTY_50 - (i * 50))
print(len(strikes))

In [ ]:
pd.to_datetime(df['expiry'], format='%d-%b-%Y', errors='coerce')
print(df['expiry'])

In [ ]:
Nifty_df = df[ (df['symbolname'] == 'NIFTY') & 
              (df['strikeprice'].isin(strikes))].sort_values(by='expiry')
# Nifty_df = Nifty_df.set_index('expiry', inplace=True)
print(Nifty_df.head())
expiries = Nifty_df['expiry'].unique()

Nifty_df = Nifty_df[Nifty_df['expiry'] == expiries[0]]
print(expiries)

In [ ]:
print(len(Nifty_df))
Nifty_df.to_csv('nifty_strikes_without_groupby.csv', index=False)

In [ ]:
Nifty_df.to_csv('nifty_strikes_with_groupby.csv', index=False)

In [1]:
import redis
r = redis.Redis(host='localhost', port=6379, db=0)

In [ ]:
import json
ltp = json.loads(r.get(f"reduced_quotes:NIFTY").decode())
print(json.dumps(ltp,indent=4))

In [3]:
import orjson

data = orjson.loads(r.get(f"reduced_quotes:NIFTY").decode())

In [4]:
print(data,'\n', type(data))

{'response': {'data': {'aPr': '0.00', 'aSz': '0', 'altt': '08 Aug 2025, 03:31:01 PM', 'avgPrc': '0.00', 'bPr': '0.00', 'bSz': '0', 'c': '24596.15', 'chg': '-232.85', 'chgp': '-0.95', 'futPrc': '0.00', 'h': '24585.50', 'hCL': '0.00', 'l': '24337.50', 'lCL': '0.00', 'ltp': '24363.30', 'ltq': '0', 'ltt': '08 Aug 2025, 03:31:01 PM', 'lut': '08 Aug 2025, 03:31:01 PM', 'mO': '0', 'ntTrdVal': '0.00', 'o': '24544.25', 'oI': '0', 'oiChg': '0.00', 'oiChgp': '--', 'spotPrc': '0.00', 'sym': '-29', 'tBQ': '0', 'tSQ': '0', 'vol': '0', 'yHgh': '26277.35', 'yLw': '21743.65'}, 'streaming_type': 'quote'}} 
 <class 'dict'>


In [ ]:
from basic_functions import common_functions



In [ ]:
obj = common_functions()
data = obj.refresh_strikes_and_options()

In [ ]:
tokens_list = list(data.keys())

In [ ]:
print(len(tokens_list))

In [ ]:
strikes_updates_channel = r.pubsub()
strikes_updates_channel.subscribe('strikes_updates')

In [ ]:
import json
for message in strikes_updates_channel.listen():
    if message['type'] == 'message':
        data = json.loads(message['data'])
        print(f"Received strikes update:{len(data)} {data}")
        # symbol = data.get('symbol')
        # if symbol:
        #     print(f"Received strikes update for {symbol}: {data}")
           